In [1]:

from sklearn.cluster import KMeans

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import LocalOutlierFactor

from category_encoders import OrdinalEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

###
### Data

In [2]:
d_tr = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/HousePrices/train.csv', index_col='Id')
d_te = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/HousePrices/test.csv', index_col='Id')

###
> **Training** and **test** sets  
> Deliting a **zero varianza** columns

In [3]:
# y = d_tr.SalePrice.copy()

# Utilities has just 2 categories, and one of them
# just appears once so we delete the whole column.
d_tr = d_tr.drop(columns='Utilities')
d_te = d_te.drop(columns='Utilities')


all_d = pd.concat([d_tr.copy(), d_te.copy()])

####
> A set of the **raw columns**

In [4]:
raw_columns = set( all_d.columns )

###
## Imputing **nan**

> columns with **nan**

In [4]:
nan_columns = all_d.columns[all_d.isna().any()].to_list()

print('number of columns that has nan: ', len(nan_columns)) #This inclued 'SalesPrice' columns

number of columns that has nan:  34


> **coustum** Functions and Transformers

In [4]:
# 34 columns with nan

def fillWithNone(df):
    ''' nan in 'PoolQC' means 'no pool' 
        nan in 'MiscFeature' means 'no misc feature'
        nan in 'Alley' means 'no alley acces'
        nan in 'Fence' means 'no fence'
        nan in 'FireplaceQu' means 'no Fireplace'
        nan in 'GarageType', 'GarageFinish', 'GarageQual',
            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1'
            'BsmtFinType1', 'MasVnrType', 'MSSubClass'
            'GarageCond' replaced with 'None' too
    
        recive a df
    '''
    
    df = df.copy()
    
    columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
               'FireplaceQu', 'GarageType', 'GarageFinish', 
               'GarageQual', 'GarageCond', 'BsmtQual',
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
               'BsmtFinType2', 'MasVnrType'
              ]
    
    for col in columns:
        df[col] = df[col].fillna('None')
    
    return df

def fillWithZero(df):
    ''' nan 
    
    '''
    
    df = df.copy()
    
    columns = ['GarageYrBlt', 'GarageArea', 'GarageCars',
               'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
               'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
               'MasVnrArea'
              ]
    
    for col in columns:
        df[col] = df[col].fillna(0)
    
    return df

def fillWithMode(df):
    ''' fill missing values with mode, median
    '''
    df = df.copy()
    
    columns = ['Electrical', 'KitchenQual', 'Exterior1st',
               'Exterior2nd', 'SaleType'
              ]
    
    for col in columns:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # filling with median of each 'Neighborhood'
    df['LotFrontage'] = (
                     df.groupby('Neighborhood')['LotFrontage']
                     .transform(lambda x: x.fillna(x.median()))
                    )  
    
    # nan means Typical
    df['Functional'] = df['Functional'].fillna('Typ')
    
    return df


def mszoning(df):
    ''' recives a DF this imputation takes place on test data only'''
    
    df = df.copy()
    
    idotrr = ( (df['Neighborhood'] == 'IDOTRR') & (df['MSZoning'].isna()) )
    mitchel = ( (df['Neighborhood'] == 'Mitchel') & (df['MSZoning'].isna()) )
    
    df.loc[ idotrr , 'MSZoning'] = 'RM'
    df.loc[ mitchel, 'MSZoning'] = 'RL'
    
#     # to test this function out of here
#     temp = mszoning(d_te)
#     # lable index acces at [1916, 2217, 2251, 2905
#     temp.loc[[1916, 2217, 2251, 2905], 'MSZoning']
    
    return df
    
none_func = FunctionTransformer(fillWithNone, validate=False) 
zero_func = FunctionTransformer(fillWithZero, validate=False) 
mode_func = FunctionTransformer(fillWithMode, validate=False)
mszo_func = FunctionTransformer(mszoning, validate=False)

> making the imputer

In [5]:
imputer = Pipeline([
                    ('withNone', none_func), 
                    ('withZero', zero_func), 
                    ('withMode', mode_func), 
                    ('mszoni', mszo_func)
                   ])

> Imputing

In [7]:
new_d = imputer.fit_transform(all_d)

> checking if there is any columns with **nan**

In [7]:
new_d.columns[new_d.isna().any()].to_list()

['SalePrice']

In [8]:
new_d.SalePrice.tail(7)

Id
2913   NaN
2914   NaN
2915   NaN
2916   NaN
2917   NaN
2918   NaN
2919   NaN
Name: SalePrice, dtype: float64

###
### Reducing low frequency categories

In [8]:
# Low frequency categories to 'Other'

def lower_than(df, col, percentage):
    ''' function that will merge low frequency classes into 
        a single class 'Others'
        
        parameters:
            df: a DataFrame
            col: column's name to work on
            percentage: the threshold 
            
        returns:
            df: the data frame with col's classes that are lowers
                than 'threshold' been repleced with 'Other' category
    '''
    
    # calculating the column frequency
    col_freq = df[col].value_counts(normalize=True)
    # the getting the column threshold 
    threshold = col_freq.quantile(q= percentage)
    # knowing the classes that are below the threshold
    less_freq_classes = col_freq[ col_freq <= threshold ]
    
    others = less_freq_classes.index.to_list()
    
    print(others)
    
    df[col] = df[col].replace(others, 'Others')
    
    return df

con1_others = ['RRAn', 'PosN', 'RRAe', 'PosA', 'RRNn', 'RRNe']
roofS_others = ['Gambrel', 'Flat', 'Mansard', 'Shed']
foun_others = ['Slab', 'Stone', 'Wood']
gara_others = ['None', 'Basment', '2Types', 'CarPort']
saleT_others = ['ConLD', 'CWD', 'ConLI', 'ConLw', 'Oth', 'Con']
saleC_others = ['Family', 'Alloca', 'AdjLand']
exte1_others = ['BrkComm', 'AsphShn', 'Stone', 'CBlock', 'ImStucc']
exte2_others = ['BrkComm', 'AsphShn', 'Stone', 'CBlock', 'ImStucc', 'Other']
lotC_others = ['FR2', 'FR3']

new_d['Condition1'] = new_d['Condition1'].map(lambda x: 'Others' if x in con1_others else x)
new_d['RoofStyle'] = new_d['RoofStyle'].map(lambda x: 'Others' if x in roofS_others else x)
new_d['Foundation'] = new_d['Foundation'].map(lambda x: 'Others' if x in foun_others else x)
new_d['GarageType'] = new_d['GarageType'].map(lambda x: 'Others' if x in gara_others else x)
new_d['SaleType'] = new_d['SaleType'].map(lambda x: 'Others' if x in saleT_others else x)
new_d['SaleCondition'] = new_d['SaleCondition'].map(lambda x: 'Others' if x in saleC_others else x)
new_d['Exterior1st'] = new_d['Exterior1st'].map(lambda x: 'Others' if x in exte1_others else x)
new_d['Exterior2nd'] = new_d['Exterior2nd'].map(lambda x: 'Others' if x in exte2_others else x)
new_d['LotConfig'] = new_d['LotConfig'].map(lambda x: 'Others' if x in lotC_others else x)


# Reducing to a BINARY CLASSES(just 2 clases)

# temp = new_d.copy()

landC_others = ['HLS', 'Bnk', 'Low']
cond2_others = ['Feedr', 'Artery', 'PosN', 'PosA', 'RRNn', 'RRAn', 'RRAe']
roofM_others = ['Tar&Grv', 'WdShake', 'WdShngl', 'Metal', 'Membran', 'Roll', 'ClyTile']
heati_others = ['GasW', 'Grav', 'Wall', 'OthW', 'Floor']
elect_others = ['FuseA', 'FuseF', 'FuseP', 'Mix']


new_d['LandContour'] = new_d['LandContour'].map(lambda x: 'Others' if x in landC_others else x)
new_d['Condition2'] = new_d['Condition2'].map(lambda x: 'Others' if x in cond2_others else x)
new_d['RoofMatl'] = new_d['RoofMatl'].map(lambda x: 'Others' if x in roofM_others else x)
new_d['Heating'] = new_d['Heating'].map(lambda x: 'Others' if x in heati_others else x)
new_d['Electrical'] = new_d['Electrical'].map(lambda x: 'Others' if x in elect_others else x)

# to encode using OneHot (17 so far)
cat_1Hot = {'Condition1': str, 'RoofStyle': str, 'Foundation': str, 'GarageType': str, 'SaleType': str, 
          'SaleCondition': str, 'Exterior1st': str, 'Exterior2nd': str, 'LotConfig': str, 'LandContour': str, 
          'Condition2': str, 'RoofMatl': str, 'Heating': str, 'Electrical': str, 'PeakMonths':str, 
          'Finished':str, 'Splited':str }


###
> Reducing number of classes in **'Neighborhood'** and **'MSSubClass'**   Using **Clusters**

In [11]:
neig_clusters = KMeans(n_clusters=5, random_state=777)
mssu_clusters = KMeans(n_clusters=4, random_state=777)

# 'Neighborhood' and 'MSSubClass' stats
neig_stats = d_tr.groupby('Neighborhood')['SalePrice'].describe()
mssu_stats = d_tr.groupby('MSSubClass')['SalePrice'].describe()

# Getting clusters
neig_clusters.fit(neig_stats)
mssu_clusters.fit(mssu_stats)

# preparing DF with cluster lables to merge
neig_df = pd.DataFrame({'Neighborhood': neig_stats.index.to_list(),
                        'Neighborhood_Cluster': neig_clusters.labels_.tolist()})

mssu_df = pd.DataFrame({'MSSubClass': mssu_stats.index.to_list(),
                        'MSSubClass_Cluster': mssu_clusters.labels_.tolist() })

In [30]:
def make_clusters(df, col, y, n_cluster):
    '''
        function to boil down a colum to n_cluster
        
        recive:
        
            df: a data frame 
            col: col to make cluster
            y: the response variable
            n_cluster: stamitation of number of cluster
            
        return:  
            a data frame with 'col' droped, and 'col' + '_cluster' column
            is added to the data frame

        
    '''
    col_clusters = KMeans(n_clusters=5, random_state=777)

    # 'Neighborhood' and 'MSSubClass' stats
    col_stats = df.groupby(col)[y].describe()

    # Getting clusters
    col_clusters.fit(col_stats)

    # preparing DF with cluster lables to merge
    col_cluster_df = pd.DataFrame( { col: col_stats.index.to_list(),
                                     col + '_Cluster': col_clusters.labels_.tolist()} )
    
    # merging the clusters with the data frame
    df = df.merge(col_cluster_df, how='left', on=col)
    
    return df.drop(columns=col)

In [29]:
# 'Neighborhood', 5, 'MSSubClass', 4
temp_cluster = make_clusters(d_tr.copy(), 'Neighborhood', 'SalePrice', 5)
temp_cluster['Neighborhood_Cluster'].value_counts()

1    523
2    443
4    238
0    143
3    113
Name: Neighborhood_Cluster, dtype: int64

###
> Features ingenering

In [9]:
# Creating a new feature 'PeakMonths', 'Unfinished' and 'Splited'
peak_moS = [5, 6, 7]
unfi_hou = ['1.5Unf', '2.5Unf']
spli_hou = ['SFoyer', 'SLvl']

new_d['PeakMonths'] = new_d['MoSold'].map(lambda x: 'Peak' if x in peak_moS else 'Normal' )
new_d['Finished'] = new_d['HouseStyle'].map(lambda x: 'no' if x in unfi_hou else 'yes') 
new_d['Splited'] = new_d['HouseStyle'].map(lambda x: 'yes' if x in spli_hou else 'no')

new_d['TotalSF'] = new_d['TotalBsmtSF'] + new_d['1stFlrSF'] + new_d['2ndFlrSF']

###
## Encoding Ordinal Categorical

In [11]:
##### ****** All writed by me ****** #####
##########################################

mapping_cat = [
    {
        'col': 'FireplaceQu',
        'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    },
    {
        'col': 'GarageQual',
        'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    },
    {
        'col': 'GarageCond',
        'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    },
    {
        'col': 'BsmtFinType1',
        'mapping': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
    },
    {
        'col': 'BsmtFinType2',
        'mapping': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
    },
    {
        'col': 'ExterQual',
        'mapping': {'Fa': 0, 'TA': 1, 'Gd': 2, 'Ex': 3}
    },
    {
        'col': 'ExterCond',
        'mapping': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
    },
    {
        'col': 'BsmtQual',
        'mapping': {'None': 0 , 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
    },
    {
        'col': 'BsmtCond',
        'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4}
    },
    {
        'col': 'PoolQC',
        'mapping': {'None': 0, 'Fa': 1, 'Gd': 2, 'Ex': 3}
    },
    {
        'col': 'HeatingQC',
        'mapping': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
    },
    {
        'col': 'KitchenQual',
        'mapping': {'Fa': 0, 'TA': 1, 'Gd': 2, 'Ex': 3}
    },
    {
        'col': 'BsmtExposure',
        'mapping': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
    },
    {
        'col': 'Functional',
        'mapping': {'Sev': 0, 'Maj2': 1, 'Maj1': 2, 'Mod': 3, 'Min2': 4, 'Min1': 5, 'Typ': 6}
    },
    {
        'col': 'GarageFinish',
        'mapping': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
    },
    {
        'col': 'Fence',
        'mapping': {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}
    },
    {
        'col': 'CentralAir',
        'mapping': {'N': 0, 'Y': 1}
    },
    {
        'col': 'PavedDrive',
        'mapping': {'N': 0, 'P': 1, 'Y': 2}
    },
    {
        'col': 'Street',
        'mapping': {'Grvl': 0, 'Pave': 1}
    },
    {
        'col': 'Alley',
        'mapping': {'None': 0, 'Grvl': 1, 'Pave': 2}
    },
    {
        'col': 'LandSlope',
        'mapping': {'Gtl': 0, 'Mod': 1, 'Sev': 2}
    },
    {
        'col': 'LotShape',
        'mapping': {'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3}
    },
    {
        'col': 'HouseStyle', 
        'mapping': {'SLvl': 0, 'SFoyer': 0, '1Story': 1, '1.5Fin': 2, 
                    '1.5Unf': 2, '2Story': 3, '2.5Unf': 4, '2.5Fin': 4}
    }
]

# to encode
# hou_sty = {'SLvl': 0, 'SFoyer': 0, '1Story': 1, '1.5Fin': 2, '1.5Unf': 2, '2Story': 3, '2.5Unf': 4, '2.5Fin': 4}
# new_d['HouseStyle'] = new_d['HouseStyle'].map(hou_sty)


# list of categorical columns(23) that are alredy done
cat_ord_DONE = ['FireplaceQu', 'GarageQual','GarageCond','BsmtFinType1',
                 'BsmtFinType2','ExterQual','ExterCond','BsmtQual',
                 'BsmtCond','PoolQC','HeatingQC','KitchenQual',
                 'BsmtExposure','Functional','GarageFinish','Fence', 'CentralAir',
                 'PavedDrive','Street','Alley','LandSlope','LotShape', 'HouseStyle']


# no work on the next columns
cat_DONE = ['OverallQual', 'OverallCond', 'MSSubClass']

# object columns (need to be encode)
cat_NoDone = ['Neighborhood', 'MSZoning', 'LotConfig', 'Condition1', 'BldgType', 'RoofStyle', 
              'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'GarageType', 'MiscFeature',
              'MoSold', 'YrSold', 'SaleType', 'SaleCondition']



In [12]:
## OrdinalEncoder
oe = OrdinalEncoder(mapping=mapping_cat)

In [13]:
temp_df = oe.fit_transform(new_d)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PeakMonths,Finished,Splited,TotalSF
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,1,0,0,Lvl,Inside,0,...,0,2,2008,WD,Normal,208500.0,Normal,yes,no,2566.0
2,20,RL,80.0,9600,1,0,0,Lvl,Others,0,...,0,5,2007,WD,Normal,181500.0,Peak,yes,no,2524.0
3,60,RL,68.0,11250,1,0,1,Lvl,Inside,0,...,0,9,2008,WD,Normal,223500.0,Normal,yes,no,2706.0
4,70,RL,60.0,9550,1,0,1,Lvl,Corner,0,...,0,2,2006,WD,Abnorml,140000.0,Normal,yes,no,2473.0
5,60,RL,84.0,14260,1,0,1,Lvl,Others,0,...,0,12,2008,WD,Normal,250000.0,Normal,yes,no,3343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,1,0,0,Lvl,Inside,0,...,0,6,2006,WD,Normal,NaN,Peak,yes,no,1638.0
2916,160,RM,21.0,1894,1,0,0,Lvl,Inside,0,...,0,4,2006,WD,Abnorml,NaN,Normal,yes,no,1638.0
2917,20,RL,160.0,20000,1,0,0,Lvl,Inside,0,...,0,9,2006,WD,Abnorml,NaN,Normal,yes,no,2448.0


In [17]:
temp_df[cat_ord_DONE].describe().T

,count,mean,std,min,25%,50%,75%,max
FireplaceQu,2919.0,1.768071,1.806619,0.0,0.0,1.0,4.0,5.0
GarageQual,2919.0,2.800959,0.715863,0.0,3.0,3.0,3.0,5.0
GarageCond,2919.0,2.808839,0.712498,0.0,3.0,3.0,3.0,5.0
BsmtFinType1,2919.0,3.541624,2.113851,0.0,1.0,4.0,6.0,6.0
BsmtFinType2,2919.0,1.273724,0.955180,0.0,1.0,1.0,1.0,6.0
ExterQual,2919.0,1.396711,0.580293,0.0,1.0,1.0,2.0,3.0
ExterCond,2919.0,2.085646,0.372361,0.0,2.0,2.0,2.0,4.0
BsmtQual,2919.0,2.505310,0.808553,0.0,2.0,3.0,3.0,4.0
BsmtCond,2919.0,2.918465,0.574950,0.0,3.0,3.0,3.0,4.0
PoolQC,2919.0,0.007537,0.135827,0.0,0.0,0.0,0.0,3.0


In [18]:
len(cat_ord_DONE)

23